# How to execute and import a jupyter notebook?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?



In [5]:
// initialize
var path = require('path');

// TLDR; What do we end up with?
var importNotebook = (notebook) => Promise.resolve({});
var getCells = ((notebook, types = ['code']) => Promise.resolve([]));
var runAllInNEL = (cells) => Promise.resolve({values: [], session: {}});
var assignResults = (results, filename) => results;

// provide a function for importing any notebook as a module and executing it
var importNotebook = (notebook, sameContext = true) => {
    var filename = path.resolve(notebook);
    var dirname = path.resolve(path.dirname(filename));
    var oldDir = process.cwd();
    var results;
    process.chdir(dirname);
    // only read javascript kernels
    return getCells(filename, ['javascript', 'code'])
    .then(cells => cells.map(c => c.source.join('')))
    .then(cells => {
        // TODOCUMENT: Notebooks are imported in to the same context so filename and dirname are overwritten.
        if (cells.length > 0) {
            cells[0] = 'var __filename='+JSON.stringify(filename)+';\n'
                + 'var __dirname='+JSON.stringify(dirname)+';\n' 
                + 'process.chdir('+JSON.stringify(dirname)+');\n' 
                + cells[0];
        }
        return cells;
    })
    .then(cells => runAllInNEL(cells, sameContext))
    .then((r) => (results = assignResults(r, notebook)))
    .then(() => {
        process.chdir(oldDir);
        return results;
    })
};
// output nothing here, cached version of this function is assigned below.
// $$.done(); for our dumb parser



What do I need to have installed?

How to sanity check the environment?


In [6]:
// initialize

var execSync = require('child_process').execSync;
var deps = ['npm install', 'JSONStream', 'nel', 'path', 'fs', 'vm'];
try {
    require.resolve('JSONStream');
    require.resolve('nel');
} catch (e) {
    execSync(deps.slice(0, 3).join(' '), {stdio: 'inherit'});
}

// $$.done() for our dumb parser


'/Users/briancullinan/jupytangular2/node_modules/nel/index.js'

How to stream json using a {match} function?

In [7]:
// initialize
var fs = require('fs');
var JSONStream = require('JSONStream');
streamJson = (file, parse, match = ((c) => true)) => {
    var file = fs.createReadStream(file)
        .pipe(JSONStream.parse(parse));
    file.on('data', (m) => match(m));
    // create a promise out of this stream
    return new Promise((resolve, reject) => {
        file.on('error', e => reject(e, file));
        file.on('close', () => resolve(file))
    });
};
streamJson;
// $$.done() for our dumb parser



[Function: streamJson]


How to parse cells of {types} from a notebook?



In [8]:
// initialize

// How to use getCells?
getCells = (notebook, types = ['*', 'code']) => {
    var cells = [];
    var kernel;
    return streamJson(notebook, [true, {emitPath: true}], (match) => {
        if(match.path[0] === 'metadata'
           && match.path[1] === 'kernelspec') {
            kernel = match.value;
        } else if(match.path[0] === 'cells'
                  && types.indexOf(match.value.cell_type) > -1) {
            cells[cells.length] = match.value;
        }
    }).then((file) => {
        if (types.indexOf(kernel.language) === -1
           && types.indexOf('*') === -1) {
            console.log('tried to import ' + types + JSON.stringify(kernel))
            return [];
        }
        return cells.map(c => Object.assign(c, {language: kernel.language}));
    });
};
getCells;

// $$.done() for our dumb parser


[Function: getCells]

How to run all promises sequentially?



In [9]:
// initialize

var runAllPromises = (promises) => {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (typeof func == 'function'
                ? (new Promise(func)) : Promise.resolve(func))
                .then(Array.prototype.concat.bind(result));
        });
    },Promise.resolve([]));
};
runAllPromises;
// $$.done() for our dumb parser



[Function: runAllPromises]

How to run all code cells in jupyter?

In [10]:
// initialize
var nel = require('nel');
// How to create VM promises out of the cell content?
var sess;
runAllInNEL = (cells, sameContext = true) => {
    if(typeof sess === 'undefined' || sameContext === false) {
        sess = new nel.Session();
    }
    return runAllPromises(
        cells.map(
            (cell) =>
            (resolve) =>
            sess.execute(cell, {
                onStdout: console.log,
                onStderr: console.error,
                onSuccess: (o) => resolve(o),
                onError: (o) => resolve(o),
                beforeRun: () => {}})
        )).then((r) => (r.session = sess) && (r));
};
runAllInNEL;

// $$.done() for our dumb parser



[Function: runAllInNEL]

In [3]:
// initialize
if (typeof imported != 'object' || imported == null) { var imported = {}; }

// display a heading when new notebooks are imported
var logIm = (cells, filename) => {
    return 'Running ' + filename  
                + ' with ' + cells.length + ' cells';
};

function returnNEL(r) {
    if(typeof r[0]['mime'] !== 'undefined' 
        && typeof r[0]['mime']['application/json'] !== 'undefined') {
        value = r[0]['mime']['application/json'];
    }
    else if(typeof r[0]['mime'] !== 'undefined' 
        && typeof r[0]['mime']['text/json'] !== 'undefined') {
        value = r[0]['mime']['text/json'];
    }
    else if (typeof r[0]['mime'] !== 'undefined'
        && typeof r[0]['mime'][Object.keys(r[0]['mime'])[0]] != 'undefined') {
        value = r[0]['mime'][Object.keys(r[0]['mime'])[0]]
    }
    else {
        value = r[0];
    }
    return value;
}

function wrapCallback (func, session) {
    // arguments doesn't work on anonymous functions like the return value:
    return (a1,a2,a3,a4,a5,a6,a7,a8,a9) => {
        var exec = '$$.async(); Promise.resolve(' 
        + func + '.apply(this,' 
        + JSON.stringify([a1,a2,a3,a4,a5,a6,a7,a8,a9]) + '))'
        + '.then(r => $$.mime({"application/json": r}))' 
        + '.catch(r => $$.mime({"application/json": r}));';
        
        return runAllInNEL([exec], session)
            .then(returnNEL);
    };
}

// cache results when notebooks are processed
(assignResults = (results, notebook) => {
    var filename = path.basename(notebook);
    var session = results.session;
    var results = results.reduce((obj, r, i) => {
        if (typeof r === 'object' && typeof r['mime'] !== 'undefined'
            && typeof r['mime']['text/plain'] !== 'undefined') {
            var re = (/\[Function:\s*(.*?)\]/ig);
            var name = re.exec(r['mime']['text/plain']);
            if(name !== null) {
                obj[filename+'['+i+']'] = wrapCallback(name[1], session);
                obj[name[1]] = obj[filename+'['+i+']'];
            }
        } else if (typeof r === 'object' 
                   && typeof r['error'] !== 'undefined') {
            obj[filename+'['+i+']'] = r['error'];
            console.log(filename+'['+i+']' + '\n' + r['error']['evalue']);
        }
        return obj;
    }, {});
    // TODO: offer results as an express service instead of global context?
    imported[filename] = results;
    return results;
});

// $$.done() for our dumb parser


[Function: assignResults]

How to import a {notebook}?

How to test the import feature?


In [12]:

// initialize
if (typeof imported != 'object' || imported == null) { var imported = {}; }
// How to test if a notebook has already been imported?
var oldImport = importNotebook, lastKeys;
(importNotebook = (notebook) => {
    var filename = path.basename(notebook);
    if(typeof notebook === 'undefined') {
        return Promise.resolve({});
    }
    if (typeof imported[filename] !== 'undefined') {
        if(lastKeys != Object.keys(imported)) {
            console.warn('Already imported');
            console.warn((lastKeys = Object.keys(imported)));
        }
        return Promise.resolve(imported[filename]);
    }
    return oldImport(notebook);
});

// $$.done() for our dumb parser


[Function: importNotebook]

How to load interpreter?

How to search notebooks?


In [2]:
// initialize
var path = require('path');
var interpretAllPath = path.join(__dirname, './interpret all notebooks.ipynb');

// save for later use
var interpret, interpretObject;
var runInNewContext = (code, ctx) => {};
var interpretAll = (queries, root = '') => {
    return (typeof interpret === 'undefined'
    ? importNotebook(interpretAllPath)
        .then((f) => {
            interpret = f['interpret'];
            interpretObject = f['interpretObject'];
        })
    : Promise.resolve())
    // use Promise.all to do all of your code searches up front
    .then(() => typeof queries == 'string' 
          ? interpret(queries) 
          : Promise.all(queries.map(interpret)))
    .then(r => typeof queries == 'string' 
          ? interpretObject([r[0]])
          : interpretObject(r.map(r => r[0])))
    .then(r => r.map(r => {
        if(typeof r === 'undefined') {
            return r;
        }
        // must have a unique id for each unique cell so that 
        // individual cells can serve as modules as well as notebooks
        // adding the cell id as a part of the filename
        r.runFresherInNewContext = (ctx) => runInNewContext(
            r.fresher,
            Object.assign(ctx || {}, {useCache: false, __filename: r.filename + r.id}),
            {filename: r.filename + r.id}, false);
        r.runInNewContext = (ctx) => runInNewContext(
            r.code,
            Object.assign(ctx || {}, {__filename: r.filename + r.id}),
            {filename: r.filename + r.id}, false);
        // TODO: specify line offset to execute service?
        r.runInNEL = (ctx) => runAllInNEL([
            'Object.assign(global, ' 
            + JSON.stringify(ctx || {}) + ');' + r.code])
            .then((results) => assignResults(results, r.notebook));
        r.runAsModule = (ctx) => runInNewContext(
            r.code,
            Object.assign(ctx || {}, {__filename: r.filename + r.id}),
            {filename: r.filename + r.id});
        return r;
    }))
    .then(r => typeof queries == 'string' ? r[0] : r);
};
interpretAll;
// $$.done() for our dumb parser



[Function: interpretAll]

Import arbitrary code in to the current context?

In [14]:
// initialize
var path = require('path');
var requireCode = (code, pathToCode, global) => {
    var Module = require("module").Module;

    // must have a new name for every generation otherwise cache will be returned
    var filepath = path.resolve(process.cwd(), pathToCode);
    var filename = path.basename(filepath);
    var dirname = path.dirname(filepath);

    var cachedModule = Module._cache[filepath];
    if (cachedModule && global.useCache !== false) {
        global.module = cachedModule;
        return cachedModule.exports;
    }

    var mod = new Module(filepath, global.module);
    global.module = mod;
    Module._cache[filepath] = mod;

    mod.filename = filepath;
    mod.paths = Module._nodeModulePaths(dirname);

    mod._compile(code, filepath);
    mod.loaded = true;

    global.exports = mod.exports;
    return mod.exports;
};
requireCode;
// $$.done() for our dumb parser



[Function: requireCode]

Import arbitrary code in to a new context?

In [15]:
// initialize
var path = require('path');
var vm = require('vm');
var runInNewContext = (code, ctx, options, isModule = true) => {
    var tmpGlobal = ctx || {};
    tmpGlobal.global = tmpGlobal;
    Object.assign(tmpGlobal, ctx);
    Object.assign(tmpGlobal, {
        requireCode: requireCode,
        moduleCode: '"use strict";\n' + code,
        projectPath: ctx.__filename,
        'console': console,
        'process': process,
        'Promise': Promise,
        'Buffer': Buffer,
        'clearImmediate': clearImmediate,
        'clearInterval': clearInterval,
        'clearTimeout': clearTimeout,
        'setImmediate': setImmediate,
        'setInterval': setInterval,
        'setTimeout': setTimeout,
        '__filename': ctx.__filename,
        '__dirname': path.dirname(ctx.__filename),
        'require': require,
    });
    Object.assign(options || {}, {lineOffset: -1});
    if(!isModule) {
        tmpGlobal.module = {};
    }
    //var result = vm.runInNewContext('"use strict";(requireCode(moduleCode, projectPath, global));', tmpGlobal);
    var result = vm.runInNewContext(isModule 
          ? 'requireCode(moduleCode, projectPath, global);'
          : tmpGlobal.moduleCode, tmpGlobal, options);
    return tmpGlobal.global.module.exports;
};
runInNewContext;
// $$.done() for our dumb parser


[Function: execInNewContext]

In [16]:
// initialize

if (typeof imported != 'object' || imported == null) { var imported = {}; }

imported['import notebook.ipynb'] = {
    'import notebook.ipynb[2]': streamJson,
    streamJson: streamJson,
    'import notebook.ipynb[3]': getCells,
    getCells: getCells,
    'import notebook.ipynb[4]': runAllPromises,
    runAllPromises: runAllPromises,
    'import notebook.ipynb[5]': runAllInNEL,
    runAllInNEL: runAllInNEL,
    'import notebook.ipynb[6]': assignResults,
    assignResults: assignResults,
    'import notebook.ipynb[7]': importNotebook,
    importNotebook: importNotebook,
    'import notebook.ipynb[8]': imported,
    imported: imported,
    'import notebook.ipynb[9]': interpretAll,
    interpretAll: interpretAll,
    'import notebook.ipynb[10]': requireCode,
    requireCode: requireCode,
    'import notebook.ipynb[11]': runInNewContext,
    runInNewContext: runInNewContext
};
imported;

// $$.done() for our dumb parser



{ 'import notebook.ipynb': 
   { 'import notebook.ipynb[2]': [Function: streamJson],
     streamJson: [Function: streamJson],
     'import notebook.ipynb[3]': [Function: getCells],
     getCells: [Function: getCells],
     'import notebook.ipynb[4]': [Function: runAllPromises],
     runAllPromises: [Function: runAllPromises],
     'import notebook.ipynb[5]': [Function: runAllInNEL],
     runAllInNEL: [Function: runAllInNEL],
     'import notebook.ipynb[6]': [Function: assignResults],
     assignResults: [Function: assignResults],
     'import notebook.ipynb[7]': [Function: importNotebook],
     importNotebook: [Function: importNotebook],
     'import notebook.ipynb[8]': [Circular],
     imported: [Circular],
     'import notebook.ipynb[9]': [Function: interpretAll],
     interpretAll: [Function: interpretAll],
     'import notebook.ipynb[10]': [Function: requireCode],
     requireCode: [Function: requireCode],
     'import notebook.ipynb[11]': [Function: execInNewContext],
     execInN

These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?

What is code completion for jupter notebooks?

